In [1]:
!apt-get update # 更新包
!apt install -y chromium-chromedriver
!pip install pillow selenium beautifulsoup4 requests ipywidgets

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,551 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,436 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [3]:
import os
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import ipywidgets as widgets
from IPython.display import display, Image as IPImage
from PIL import Image as PILImage, ImageDraw
from io import BytesIO
from urllib.parse import urljoin
import io
import matplotlib.pyplot as plt

# 配置 Chrome 选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 在无头模式下运行
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 初始化 WebDriver
# service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(options=chrome_options) # service=service,

def cut_image_bottom(image, cut_value):
    print(f"cut_image_bottom， 裁剪底部{cut_value}")
    # 确保裁剪的高度不超过原始高度
    if cut_value < 0:
        raise ValueError("裁剪高度不能大于图像高度")
     # 裁剪图片
    width, height = image.size
    cropped_image = image.crop((0, 0, width, height - cut_value))
    return cropped_image

def remove_unwanted_characters(input_string):
    unwanted_characters = ['%', '；', '？', ';', '?', '^', '@', '!', '&', '*', '(', ')', '~', ':', '=']

    for char in unwanted_characters:
        input_string = input_string.replace(char, '')

    return input_string

# 定义函数以从网址抓取所有图片
def fetch_images_from_url(url):
    saved_files = []

    # 获取页面
    driver.get(url)
    # 等待页面加载，调整时间以确保页面加载完成
    sleep(10)  # 您可以根据需要调整等待时间
    # 现在获取页面的 HTML
    html_content = driver.page_source

    # response = requests.get(url)
    # soup = BeautifulSoup(response.text, 'html.parser')
    soup = BeautifulSoup(html_content, 'html.parser')

    # 根据实际情况，这里选择包含文章内容的元素（如 <article>、<div class="content"> 等）
    article = soup.find('article')  # 获取文章内容

    # 确保找到了文章内容
    if article is None:
      print("找不到指定的文章内容。请检查选择器是否正确。")
    else:
      print(f"title: {title}")

    # 创建一个目录来保存下载的图片
    if not os.path.exists(title):
        os.makedirs(title, exist_ok=True)

    # 找到所有的 <img> 标签
    images = article.find_all('img')
    index = 0
    # 下载每个图片
    for img_item in images:
        index += 1
        img_url = img_item.get('src')
        # 确保 img_url 是完整的 URL
        img_url = urljoin(url, img_url)
        try:
          # 获取图片数据
          img_response = requests.get(img_url)
          # 检查响应内容类型
          content_type = img_response.headers['Content-Type']
          # 根据 MIME 类型确定扩展名
          if 'image/jpeg' in content_type:
              extension = '.jpg'
          elif 'image/png' in content_type:
              extension = '.png'
          elif 'image/gif' in content_type:
              extension = '.gif'
          elif 'image/webp' in content_type:
              extension = '.webp'
          else:
              print(f"不支持的图片类型: {content_type}")
              continue  # 跳过不支持的图片类型

          # 获取图片名称并添加扩展名
          img = PILImage.open(BytesIO(img_response.content))
          if (cut_bottom > 0):
              img = cut_image_bottom(img, cut_bottom)
          if 'image/webp' in content_type:
              print(f"把{content_type}转成jpeg格式")
              # 转换为 JPEG 格式
              # img_name = (img_url.split('/')[-1] + extension).replace('.webp', '.jpg')
              img_name = f"{title}_{index}.jpg"
              # img.convert('RGB').save(img_name, 'JPEG')
          else:
              # img_name = img_url.split('/')[-1] + extension
              img_name = f"{title}_{index}{extension}"

          # 除掉img_name的不正规字符
          print(f'除掉字符之前: {img_name}')
          img_name = remove_unwanted_characters(img_name)

          img_name_with_onedir = (os.path.join(title, img_name))
          print(f'保存图片: {img_name_with_onedir}')
          # 保存图片
          # img.save(img_name_with_onedir, 'JPEG')  # .convert('RGB')
          img.convert('RGB').save(img_name_with_onedir, 'JPEG', quality=95, optimize=True)

          saved_files.append(img_name_with_onedir)
        except Exception as e:
          print(f'执行失败: {img_url}，错误：{e}')

    return saved_files

# 创建占位图像
def create_placeholder_image(size=(8, 6), color=(200, 200, 200)):
    """ 创建一个指定颜色的占位图像 """
    img = PILImage.new("RGB", (size[0]*100, size[1]*100), color)  # 乘以100是为了适应Matplotlib的显示
    draw = ImageDraw.Draw(img)
    draw.text((10, 10), "正在加载...", fill=(0, 0, 0))  # 添加一些文字
    return img

# 定义函数以显示图像
class ImageViewer:
    def __init__(self, images):
        self.images = images
        self.index = 0

        self.prev_button = widgets.Button(description='上一张')
        self.next_button = widgets.Button(description='下一张')

        self.placeholder_image = create_placeholder_image()  # 创建占位图像

        self.image_display = widgets.Output()  # 用于显示图像

        self.prev_button.on_click(self.show_prev_image)
        self.next_button.on_click(self.show_next_image)

        self.update_buttons()  # 更新按钮状态
        display(self.prev_button, self.next_button, self.image_display)
        self.show_image()  # 显示第一张图片

    def show_image(self):
        # 使用 PIL 打开并显示图像
        img_path = self.images[self.index]

        with self.image_display:
            self.image_display.clear_output()  # 清空之前的输出
            # 显示占位图像
            plt.imshow(self.placeholder_image)
            plt.axis('off')  # 关闭坐标轴
            plt.show()

        # 加载实际图片
        img = PILImage.open(img_path)

        with self.image_display:
            self.image_display.clear_output()  # 清空之前的输出
            plt.imshow(img)
            plt.axis('off')  # 关闭坐标轴
            plt.show()

        self.update_buttons()  # 更新按钮状态

    def show_prev_image(self, b):
        self.index = (self.index - 1) % len(self.images)
        self.show_image()

    def show_next_image(self, b):
        self.index = (self.index + 1) % len(self.images)
        self.show_image()

    def update_buttons(self):
        # 更新按钮的启用状态
        self.prev_button.disabled = (self.index == 0)
        self.next_button.disabled = (self.index == len(self.images) - 1)

#@markdown 要爬取图片的网页
cut_bottom = 80 #@param {type:"number"}
title = "lifuTrainTest" #@param {type:"string"}
url = "https://www.toutiao.com/article/7445892488826569242/?log_from=50f7478aad7a9_173390023253"  #@param {type:"string"}
saved_images = fetch_images_from_url(url)

# 显示图片
if saved_images:
    viewer = ImageViewer(saved_images)
else:
    print("没有找到图片。")

# 关闭 WebDriver
driver.quit()

title: lifuTrainTest
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_1.jpg
保存图片: lifuTrainTest/lifuTrainTest_1.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_2.jpg
保存图片: lifuTrainTest/lifuTrainTest_2.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_3.jpg
保存图片: lifuTrainTest/lifuTrainTest_3.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_4.jpg
保存图片: lifuTrainTest/lifuTrainTest_4.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_5.jpg
保存图片: lifuTrainTest/lifuTrainTest_5.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_6.jpg
保存图片: lifuTrainTest/lifuTrainTest_6.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_7.jpg
保存图片: lifuTrainTest/lifuTrainTest_7.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_8.jpg
保存图片: lifuTrainTest/lifuTrainTest_8.jpg
cut_image_bottom， 裁剪底部80
除掉字符之前: lifuTrainTest_9.jpg
保存图片: lifuTrainTest/lifuTrainTest_9.jpg


Button(description='上一张', disabled=True, style=ButtonStyle())

Button(description='下一张', style=ButtonStyle())

Output()